In [2]:
import csv
import pickle
from collections import defaultdict

In [3]:
with open("C:\\Users\\mapyredd\\Documents\\marco\\data\\model\\bm25.pkl", 'rb') as fr:
    bm25 = pickle.load(fr)

C:\Users\mapyredd\AppData\Local\Continuum\anaconda2\envs\deeplearning\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
import sys
from importlib import reload
import logging
import csv
from nltk.stem.porter import PorterStemmer
import string
import gensim
import pickle
from gensim.summarization.bm25 import BM25


porterStemmer = PorterStemmer()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def removeNonAscii(s):
    return "".join(filter(lambda x:ord(x) < 128, s))


def preprocess(passage):
    clean_passage = passage.lower()
    clean_passage = removeNonAscii(clean_passage)
    clean_passage = clean_passage.translate(str.maketrans('', '', string.punctuation)) 
    
    clean_passage_list = clean_passage.split()
    clean_passage_list_stem = []
    for word in clean_passage_list:
        if word not in gensim.parsing.preprocessing.STOPWORDS:
            clean_passage_list_stem.append(porterStemmer.stem(word))
    return clean_passage_list_stem


In [11]:

query_file = open("C:\\Users\\mapyredd\\Documents\\marco\\data\\queries.eval.tsv",encoding='utf-8')
query_reader = csv.reader(query_file, delimiter = "\t")

queryid_text_dict = defaultdict(str)
for row in query_reader:
    queryid_text_dict[int(row[0])] = row[1]
    
print (len(queryid_text_dict))

12560


In [6]:
count = 0

fr = open("C:\\Users\\mapyredd\\Documents\\marco\\data\\top1000.eval.tsv", encoding="UTF8")
rd = csv.reader(fr, delimiter='\t')
query_id_doc_id = defaultdict(list)
for row in rd:
    query_id = int(row[0])
    doc_id = int(row[1])
    query_id_doc_id[query_id].append(doc_id)
    count += 1
    if count % 100000 == 0:
        print(count)

    

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
0


In [13]:
average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())
count = 0

qrel_top_file = open("C:\\Users\\mapyredd\\Documents\\marco\\data\\top15.eval.tsv",'w')
for query_id, doc_ids in query_id_doc_id.items():
    docid_scores = []
    query_text = preprocess(queryid_text_dict[query_id])
    for doc_id in doc_ids:
        score = bm25.get_score(query_text, int(doc_id), average_idf)
        docid_scores.append((int(doc_id), score))
    docid_scores = sorted(docid_scores, key=lambda x:x[1], reverse=True)[:15]

    for doc_score in docid_scores:
        qrel_top_file.write(str(query_id) + "\t" + str(doc_score[0]) + "\t" + queryid_text_dict[query_id] + "\n")
    count += 1
    if count %1000 == 0:
        print (count)

1000
2000
3000
4000
5000
6000
